#◇Reference

Hugging Face Whisper:</br>
https://huggingface.co/docs/transformers/model_doc/whisper

##Install Tranformers and datasets

In [ ]:
!pip install tensorflow==2.14
#!pip install transformers
!pip install datasets

# Set up and prepair sample data

In [ ]:
import tensorflow as tf
from transformers import WhisperProcessor, TFWhisperForConditionalGeneration
from datasets import load_dataset
import os
import json
import shutil
import google.colab as colab
import numpy as np

model_name = 'whisper-tiny.en'
model_path = "openai/%s" %(model_name)
saved_model_dir = '/content/tf_whisper_saved'

work_dir = '/content/work'
os.makedirs(work_dir, exist_ok=True)
tflite_model_path = os.path.join(work_dir, "%s.tflite" %(model_name))

processor = WhisperProcessor.from_pretrained(model_path)
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
input_features = processor(ds[0]["audio"]["array"], return_tensors="tf").input_features
input_features

In [ ]:
dat = processor(ds[0]["audio"]["array"], return_tensors="tf").input_features

In [ ]:
ds[0]["audio"]["array"]

## Define generation-enabled TF model

In [ ]:
class GenerateModel(tf.Module):
  def __init__(self, model, max_new_tokens=223):
    super(GenerateModel, self).__init__()
    self.model = TFWhisperForConditionalGeneration.from_pretrained(model)
    self.max_new_tokens = max_new_tokens

  @tf.function(
    # shouldn't need static batch size, but throws exception without it (needs to be fixed)
    input_signature=[
      tf.TensorSpec((1, 80, 3000), tf.float32, name="input_features"),
    ],
  )
  def generate(self, input_features):
    outputs = self.model.generate(
      input_features,
      max_new_tokens=self.max_new_tokens, #change as needed
      return_dict_in_generate=True,
    )
    return {"sequences": outputs["sequences"]}

##Load pre trained TF Whisper Tiny model and evaluate it

In [ ]:
model = GenerateModel(model_path)

In [ ]:
generated_ids = model.generate(input_features)["sequences"]
print(generated_ids)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription

In [ ]:
tf.saved_model.save(model, saved_model_dir, signatures={"serving_default": model.generate})

##Convert saved model to TFLite model

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the tflite model (flatbuffers)
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

#Evaluate TFLite model

In [ ]:
interpreter = tf.lite.Interpreter(tflite_model_path)

tflite_generate = interpreter.get_signature_runner()
generated_ids = tflite_generate(input_features=input_features)['sequences']
print(generated_ids)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription

#Save Vocablaries: Decoder

In [ ]:
def save_vocab(save_directory: str, processor: WhisperProcessor) -> str:
  # save vocablary
  vocab_file = os.path.join(save_directory, "vocab.json")
  with open(vocab_file, "w", encoding="utf-8") as f:
    json.dump(processor.tokenizer.encoder, f, indent=2, sort_keys=True, ensure_ascii=False)

  # save added_vocablary
  added_vocab_file = os.path.join(save_directory, "added_vocab.json")
  with open(added_vocab_file, "w", encoding="utf-8") as f:
    json.dump(processor.tokenizer.added_tokens_encoder, f, indent=2, sort_keys=True, ensure_ascii=False)

  # save all_special_ids
  all_special_ids_file = os.path.join(save_directory, "special_ids.json")
  with open(all_special_ids_file, "w", encoding="utf-8") as f:
    json.dump(processor.tokenizer.all_special_ids, f, indent=2, sort_keys=True, ensure_ascii=False)

  return vocab_file, added_vocab_file, all_special_ids_file

In [ ]:
save_vocab(work_dir, processor)

#Download TFLite model

In [ ]:
shutil.make_archive(model_name, format="zip", root_dir=work_dir)
colab.files.download(model_name + ".zip")